In [ ]:
# Importing the required modules
import pymongo
import json

In [ ]:
# Creating a new database 
client = pymongo.MongoClient("mongodb://localhost:27017/")
mydb = client['Assgdb']

In [ ]:
# Creating new directory
mycoll = mydb['Students']

In [ ]:
# Loading or updating json file
l = []
with open('students.json') as f:
    for i in f:
        l.append(i)

In [ ]:
# Inserting the students record into the collection
for i in l:
    s = json.loads(i)
    mycoll.insert_one(s)

In [ ]:
# Screening all records which inserted
for e in mycoll.find():
    print(e)

In [ ]:
# Some of the list of queries that i did here is given below cells!

In [ ]:
# 1) Find the student name who scored maximum scores in all (exam, quiz and homework)?
cursor = mycoll.aggregate([
  { "$unwind": "$scores" },
  { "$group": {
      "_id": "$scores.type",
      "nameScores": {
        "$push": { "name": "$name", "score": "$scores.score" }
      },
      "max": { "$max": "$scores.score" }
    }
  },
  { "$set": {
      "max_score": {
        "$first": {
          "$filter": {
            "input": "$nameScores",
            "cond": { "$eq": [ "$$this.score", "$max" ] }
          }
        }
      }
    }
  },
  { "$project": {
      "_id": 0,
      "type": "$_id",
      "max_score": 1
    }
  }
])

# To display the result
for document in cursor:
    print(document)

In [ ]:
# 2) Find students who scored below average in the exam and pass mark is 40%?
cursor = avg_coll.aggregate([{'$unwind':'$scores'},{'$group':{'_id':'$total','Pass_student':{'$total':{'$gt':40}}}}])
for i in cursor:
    print(i)

In [ ]:
# 3) Find students who scored below pass mark and assigned them as fail, and above pass mark as pass in all the categories.
cursor = avg_coll.aggregate([{'$unwind':'$scores'},
                             {'$group':{'_id':'$_id',
                                        'status':{'$switch':
                                                  {'branches':
                                                   [{'case':{'$gt':['total',40]},'then':"pass"},
                                                   {'case':{'$lt':['total',40]},'then':"fail"}]}}}}])
for i in cursor:
    print(i)

In [ ]:
# 4) Find the total and average of the exam, quiz and homework and store them in a separate collection.
cursor = mycoll.aggregate([{ '$unwind' : "$scores"},{ '$group':{'_id': "$name",'total': { '$avg': "$scores.score" }}}])
avg_coll = mydb['Average_marks1']
for i in cursor:
    avg_coll.insert_one(i)
# To display the output
for i in avg_coll.find():
    print(i)  

In [ ]:
# 5) Create a new collection which consists of students who scored below average and above 40% in all the categories.
cursor = avg_coll.aggregate([{'$unwind':'$scores'},{'$group':{'_id':'$_id','Pass_student':{'$total':{'$gt':40}}}}])
pass_coll = mydb['pass']
for i in cursor:
    pass_coll.insert_one(i)
cursor = avg_coll.aggregate([{'$unwind':'$scores'},{'$group':{'_id':'$total','Pass_student':{'$total':{'$lt':40}}}}])
fail_coll = mydb['fail']
for i in cursor:
    fail_coll.insert_one(i)

In [ ]:
# 6) Create a new collection which consists of students who scored below the fail mark in all the categories.

In [ ]:
# 7) Create a new collection which consists of students who scored above pass mark in all the categories.